# ESL by GWL

Notebook environment to migrate csv files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [4]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math
import itertools
import glob

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))
sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata')

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("11208003-latedeo2022","020_InternationalDeltaPortfolio")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

In [22]:
os.path.dirname(cwd)

'C:\\SnapVolumesTemp\\MountPoints\\{45c63495-0000-0000-0000-100000000000}\\{548BD435-0C48-4314-A26F-734F82A3D043}\\SVROOT\\Users\\kras\\Documents\\GitHub\\global-coastal-atlas\\stac_folder'

In [5]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath(r"04_extreme_sea_levels_at_different_global_warming_levels\5DeltasESL")
dataset_dir_path = dataset_dir.joinpath("ESLbyGWL.nc")
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

### Write CSV to NetCDF

In [6]:
# write csv to netcdf

# open all csv files in different dirs
all_files = []
for dir in os.listdir(dataset_dir):
    if '.' not in dir: # arbitrary, no file extension to determine whether it is dir
        all_files.append(glob.glob(os.path.join(dataset_dir , dir,  "*.CSV")))

# read csv and convert to nc files
li = []
for filename in list(itertools.chain(*all_files)):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

    ds = xr.Dataset.from_dataframe(df)
    #ds.to_netcdf(filename.replace('.CSV', '.nc'))

# make one dataframe
#df = pd.concat(li, axis=0, ignore_index=True)

# Convert the pandas dataframe to an xarray dataset
#ds = xr.Dataset.from_dataframe(df)

# Write the xarray dataset to a netCDF file
#ds.to_netcdf(dataset_dir_file)

### Check CF compliancy original NetCDF files

In [7]:
# open datasets (only first file, rest is the same)
ds = xr.open_dataset(list(itertools.chain(*all_files))[0].replace('.CSV', '.nc'))

# check original dataset
ds

<xarray.Dataset>
Dimensions:     (index: 41)
Coordinates:
  * index       (index) int64 0 1 2 3 4 5 6 7 8 9 ... 32 33 34 35 36 37 38 39 40
Data variables: (12/14)
    longitude   (index) float64 ...
    latitude    (index) float64 ...
    Lcurrent    (index) float64 ...
    Ccurrent    (index) float64 ...
    Ucurrent    (index) float64 ...
    L1p5degree  (index) float64 ...
    ...          ...
    L3p0degree  (index) float64 ...
    C3p0degree  (index) float64 ...
    U3p0degree  (index) float64 ...
    L5p0degree  (index) float64 ...
    C5p0degree  (index) float64 ...
    U5p0degree  (index) float64 ...

In [8]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=list(itertools.chain(*all_files))[0].replace('.CSV', '.nc'), 
                 working_dir=CF_dir
                 )

In [9]:
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=list(itertools.chain(*all_files))[0].replace('.CSV', '.nc'), working_dir=CF_dir)

{'P:\\11208003-latedeo2022\\020_InternationalDeltaPortfolio\\04_extreme_sea_levels_at_different_global_warming_levels\\5DeltasESL\\Bangladesh\\ESL_100yr.nc': {'warnings': '17', 'errors': '0'}}


### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [10]:
# open datasets

ds_list = []
for i in list(itertools.chain(*all_files)):
    ds_list.append(xr.open_dataset(i.replace('.CSV', '.nc')))

In [11]:
import json

# NetCDF attribute alterations by means of metadata template
f_global = open(r'p:\11208003-latedeo2022\020_InternationalDeltaPortfolio\04_extreme_sea_levels_at_different_global_warming_levels\metadata_ESLbyGWL.json')
meta_global = json.load(f_global)

for i in ds_list:
    for attr_name, attr_val in meta_global.items():
        if attr_name == 'PROVIDERS':
            attr_val = json.dumps(attr_val)
        i.attrs[attr_name] = attr_val

    i.attrs['Conventions'] = "CF-1.8"

In [12]:
# NetCDF variable and dimension alterations (per dataset)

ds_list_CF = []
for i, j in zip(ds_list, list(itertools.chain(*all_files))):

    # rename or swap dimension names, the latter in case the name already exists as coordinate
    i = i.rename_dims({"index": "nstations"})

    # rename variables, if necessary
    i = i.rename_vars(
        {"longitude": "lon", "latitude": "lat"} #"index":"nstations", 
    )

    # set some data variables to coordinates to avoid duplication of dimensions in later stage
    i = i.set_coords(["lon", "lat"])

    # add variable esl and reshape to [gwl, ensemble, stations]
    data_array = np.concatenate([i['Lcurrent'].values, i['Ccurrent'].values, i['Ucurrent'].values, 
                                 i['L1p5degree'].values, i['C1p5degree'].values, i['U1p5degree'].values,
                                 i['L3p0degree'].values, i['C3p0degree'].values, i['U3p0degree'].values,
                                 i['L5p0degree'].values, i['C5p0degree'].values, i['U5p0degree'].values])
    data_array_r = data_array.reshape((4,3,len(i['Lcurrent'].values)))

    # remove variables
    i = i.drop(["index", "Lcurrent", "Ccurrent", "Ucurrent", "L1p5degree", "C1p5degree", "U1p5degree", "L3p0degree", "C3p0degree", "U3p0degree", "L5p0degree", "C5p0degree", "U5p0degree"])
    
    # add dimensions
    i = i.expand_dims(dim={"ensemble": [5, 50, 95]})
    i = i.expand_dims(dim={"gwl": [0, 1.5, 3, 5]})

    # add variable
    i = i.assign(esl=(["gwl", "ensemble", "nstations"], data_array_r))

    # add or change certain variable / coordinate attributes
    dataset_attributes = {
        "gwl": {"long_name": "global warming level", "units": "degrees"},
        "esl": {"long_name": "extreme sea level", "units": "m"},
        "ensemble": {"long_name": "ensemble", "units": "1"}, # set to 1 if no unit
        "lon": {"standard_name": "longitude", "long_name": "longitude", "units": "degrees_east"},
        "lat": {"standard_name": "longitude", "long_name": "latitude", "units": "degrees_north"},
    }  # specify custom (CF convention) attributes

    # add / overwrite attributes
    for k, v in dataset_attributes.items():
        try:
            i[k].attrs = dataset_attributes[k]
        except:
            continue

    #i.to_netcdf(path=str(j).replace(".CSV", "_CF.nc")) # save single CF compliant files

    ds_list_CF.append(i)

In [13]:
# merging information, pre-work to concatenate after

ds_merged_list_CF = [] # split per area and return period
ds_list_area = []
ds_list_rp = []
for idx, (i, j) in enumerate(zip(ds_list_CF, list(itertools.chain(*all_files)))):
    area = list(itertools.chain(*all_files))[idx].split("\\")[-2]
    rp = list(itertools.chain(*all_files))[idx].split("_")[-1].split("yr")[0]

    if idx == 0: # start with empty list
        area_list = []
        rp_list = []
        ds_merged_list = []

    # append information
    area_list.append(area)
    rp_list.append(float(rp))
    ds_merged_list.append(i)

    if idx+1 == len(list(itertools.chain(*all_files))): # avoid error
        ds_list_area.append(np.unique(area_list)) # append
        ds_list_rp.append(rp_list) # append
        ds_merged_list_CF.append(ds_merged_list) # append
        continue

    if list(itertools.chain(*all_files))[idx+1].split("\\")[-2] != area: # append and make new list if new area is encountered
        ds_list_area.append(np.unique(area_list)) # append
        ds_list_rp.append(rp_list) # append
        ds_merged_list_CF.append(ds_merged_list) # append
        area_list = []
        rp_list = []
        ds_merged_list = []

In [14]:
# concat datasets along rp and area to make one file with a meaningfull variable

# merge return periods
ds_merged_list_CF_rp = []
for i, j, k in zip(ds_merged_list_CF, ds_list_area, ds_list_rp): # concat for RP in loop
        #print(i, j, k)
        sorted_i = [i[l] for l in np.argsort(k)] # make monotonically increasing by indices of sorted list of rps
        flat_area_list = [j[0] for m in range(len(sorted_i[0].nstations.values))]

        rpm = xr.concat(sorted_i, dim="rp") # concatenate RPs
        rpm = rpm.assign(rp=(["rp"], np.sort(k))) # assign RP as coordinate value, sort monotonically increasing 
        rpm = rpm.assign(stations=(["nstations"], np.array(flat_area_list, dtype="S"))) # assign area variable value, note the S type as CF cannot handle objects
        rpm = rpm.set_coords("stations") # move too coordinate value

        # add or change certain variable / coordinate attributes
        dataset_attributes = {
                "rp": {"long_name": "return period", "units": "yr"},
                "stations": {"long_name": "stations", "units": "1"}, # set to 1 if no unit
        }  # specify custom (CF convention) attributes

        # add / overwrite attributes
        for k, v in dataset_attributes.items():
                try:
                        rpm[k].attrs = dataset_attributes[k]
                except:
                        continue

        ds_merged_list_CF_rp.append(rpm)

# merge areas
ds_merged_list_CF_rp_area = xr.concat(ds_merged_list_CF_rp, dim="nstations") 

In [15]:
ds_merged_list_CF_rp_area

<xarray.Dataset>
Dimensions:   (gwl: 4, ensemble: 3, nstations: 230, rp: 5)
Coordinates:
  * gwl       (gwl) float64 0.0 1.5 3.0 5.0
  * ensemble  (ensemble) int64 5 50 95
    lon       (nstations) float64 89.5 89.56 89.91 90.5 ... 8.613 8.613 8.848
    lat       (nstations) float64 22.0 22.0 21.85 23.05 ... 55.34 55.44 54.4
  * rp        (rp) float64 5.0 10.0 20.0 50.0 100.0
    stations  (nstations) |S15 b'Bangladesh' ... b'North Sea Basin'
Dimensions without coordinates: nstations
Data variables:
    esl       (rp, gwl, ensemble, nstations) float64 0.9273 0.9293 ... 6.832
Attributes: (12/21)
    TITLE:               ESL by GWL
    TITLE_ABBREVIATION:  esl_gwl
    DESCRIPTION:         Extreme sea level projections for five return period...
    SHORT_DESCRIPTION:   Extreme sea level projections for different global w...
    INSTITUTION:         Joint Global Change Research Institute, Pacific Nort...
    PROVIDERS:           {"name": "Joint Global Change Research Institute, Pa...
    ...                  ...
    DOI:                 https://doi:10.1038/s41558-021-01127-1
    LONG_NAME:           ESL_BY_GWL
    UNITS:               m
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

In [16]:
# write to NetCDF file to check compliancy

# prevent file locking, see: https://github.com/pydata/xarray/issues/2376
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

ds_merged_list_CF_rp_area.to_netcdf(path=str(dataset_dir_path).replace(".nc", "_CF.nc"))

### Check CF compliancy altered NetCDF files

In [17]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_dir_path).replace(".nc", "_CF.nc"), working_dir=CF_dir)

In [18]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=str(dataset_dir_path).replace(".nc", "_CF.nc"),
    working_dir=CF_dir,
)

{'P:\\11208003-latedeo2022\\020_InternationalDeltaPortfolio\\04_extreme_sea_levels_at_different_global_warming_levels\\5DeltasESL\\ESLbyGWL_CF.nc': {'warnings': '0', 'errors': '0'}}


In [19]:
# open the output file again to check..
#ds = xr.open_dataset(r"p:\11208003-latedeo2022\020_InternationalDeltaPortfolio\04_extreme_sea_levels_at_different_global_warming_levels\5DeltasESL\ESLbyGWL_5CF.nc")

### Write data to Zarr files

In [20]:
# export to Zarr in one-liner (as rp is the temporal dimension)

# export to zarr in write mode (to overwrite if exists)
ds_merged_list_CF_rp_area.to_zarr(str(dataset_dir_path).replace(".nc", ".zarr"), mode="w")